# Imports and Colab Mount

In [1]:
import datetime
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob
from math import sqrt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout, GRU, SimpleRNN, Input, LSTM, Dense, Bidirectional, Concatenate, Reshape, Lambda, Bidirectional
from keras.models import Model, Sequential
from keras import backend as K
from tensorflow.keras import layers
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import seaborn as sns

from numpy.random import seed
#from tensorflow import set_random_seed

%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
train_h = pd.read_csv("/content/drive/MyDrive/Datasets/solar_train.csv", index_col=0, parse_dates=True)
valid_h = pd.read_csv("/content/drive/MyDrive/Datasets/solar_valid.csv", index_col=0, parse_dates=True)
test_h = pd.read_csv("/content/drive/MyDrive/Datasets/solar_test.csv", index_col=0, parse_dates=True)
hourly = pd.read_csv("/content/drive/MyDrive/Datasets/solar_all.csv", index_col=0, parse_dates=True)

In [4]:
hourly

,PV Power,Temp_Ar_Avg,Umid_Ar_Avg,Vel_Vento,Dir_Vento,SD1_Dir,Vel_Vento_Max,Vel_Vento_Min,Vel_Vento_Std,Press_Atm_Avg,...,Irradiancia_Min,Irradiancia_Std,Irradiancia_2_Avg,Irradiancia_2_Max,Irradiancia_2_Min,Irradiancia_2_Std,Month,Year,Day,Hour
2018-07-01 06:00:00,263.030026,18.0350,82.750000,1.872250,106.325000,21.395000,2.87675,1.009000,0.49325,929.25,...,11.71475,0.23525,12.626,13.338500,12.336000,0.304500,7.0,2018.0,1.0,6.0
2018-07-01 07:00:00,1553.226240,18.7450,79.705000,1.308000,117.450000,33.175000,2.27750,0.539500,0.48950,930.00,...,170.72500,1.19675,176.950,178.725000,175.050000,1.201750,7.0,2018.0,1.0,7.0
2018-07-01 08:00:00,2933.365412,20.6575,70.537500,2.363750,106.375000,29.895000,3.47600,1.478500,0.55475,931.00,...,427.30000,1.28275,433.850,436.300000,431.650000,1.332000,7.0,2018.0,1.0,8.0
2018-07-01 09:00:00,3940.915100,21.8975,62.450000,2.780000,115.400000,26.415000,4.07500,1.478500,0.64500,931.00,...,201.75000,108.35425,373.050,554.150000,201.300000,108.448500,7.0,2018.0,1.0,9.0
2018-07-01 10:00:00,3488.088536,22.8950,57.725000,3.552250,146.700000,18.677500,4.87375,2.677000,0.57675,931.00,...,471.35000,88.83275,543.650,676.075000,469.200000,88.563000,7.0,2018.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-31 13:00:00,4804.056065,28.2225,38.147500,2.440250,167.950000,23.577500,3.67575,1.278750,0.58750,930.00,...,677.92500,2.68950,670.300,675.575000,664.475000,2.770250,7.0,2019.0,31.0,13.0
2019-07-31 14:00:00,6048.575835,28.7675,35.367500,1.724500,136.350000,29.392500,2.87675,1.079000,0.55850,928.75,...,895.20000,40.61850,925.550,992.750000,873.500000,39.891000,7.0,2019.0,31.0,14.0
2019-07-31 15:00:00,3325.500841,29.1650,33.697500,2.269250,128.675000,24.887500,3.67575,1.009000,0.63550,928.00,...,382.22500,118.95000,556.075,711.025000,372.050000,115.160250,7.0,2019.0,31.0,15.0
2019-07-31 16:00:00,2202.673578,28.7300,33.600000,1.976250,141.350000,12.645000,2.87675,1.009000,0.43100,928.00,...,314.07500,4.12300,309.925,315.175000,301.500000,4.098750,7.0,2019.0,31.0,16.0


In [8]:
values = hourly.values
values = values.astype('float32')

reframed = series_to_supervised(values, 1, 1)

In [10]:
reframed.iloc[: , :-38]

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var31(t-1),var32(t-1),var33(t-1),var34(t-1),var35(t-1),var36(t-1),var37(t-1),var38(t-1),var39(t-1),var1(t)
1,263.030029,18.035000,82.750000,1.87225,106.324997,21.395000,2.87675,1.00900,0.49325,929.25,...,0.235250,12.626000,13.338500,12.336000,0.304500,7.0,2018.0,1.0,6.0,1553.226196
2,1553.226196,18.745001,79.705002,1.30800,117.449997,33.174999,2.27750,0.53950,0.48950,930.00,...,1.196750,176.949997,178.725006,175.050003,1.201750,7.0,2018.0,1.0,7.0,2933.365479
3,2933.365479,20.657499,70.537498,2.36375,106.375000,29.895000,3.47600,1.47850,0.55475,931.00,...,1.282750,433.850006,436.299988,431.649994,1.332000,7.0,2018.0,1.0,8.0,3940.915039
4,3940.915039,21.897499,62.450001,2.78000,115.400002,26.415001,4.07500,1.47850,0.64500,931.00,...,108.354248,373.049988,554.150024,201.300003,108.448502,7.0,2018.0,1.0,9.0,3488.088623
5,3488.088623,22.895000,57.724998,3.55225,146.699997,18.677500,4.87375,2.67700,0.57675,931.00,...,88.832748,543.650024,676.075012,469.200012,88.563004,7.0,2018.0,1.0,10.0,4550.804199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,6915.987793,27.262501,42.895000,2.35350,129.375000,42.259998,3.27625,1.47850,0.52425,931.50,...,8.773250,875.174988,885.075012,853.250000,8.567750,7.0,2019.0,31.0,12.0,4804.056152
4016,4804.056152,28.222500,38.147499,2.44025,167.949997,23.577499,3.67575,1.27875,0.58750,930.00,...,2.689500,670.299988,675.575012,664.474976,2.770250,7.0,2019.0,31.0,13.0,6048.575684
4017,6048.575684,28.767500,35.367500,1.72450,136.350006,29.392500,2.87675,1.07900,0.55850,928.75,...,40.618500,925.549988,992.750000,873.500000,39.890999,7.0,2019.0,31.0,14.0,3325.500732
4018,3325.500732,29.165001,33.697498,2.26925,128.675003,24.887501,3.67575,1.00900,0.63550,928.00,...,118.949997,556.075012,711.025024,372.049988,115.160248,7.0,2019.0,31.0,15.0,2202.673584


In [5]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
def make_data(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = 39
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :-38]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:2184]
    valid_y = resid_train.values[2184:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y

In [ ]:
def make_data2(data, timestep, resid_check=False):
  values = data.values
  values = values.astype('float32')
  #scaler = MinMaxScaler(feature_range=(0, 1))
  #scaled = scaler.fit_transform(values)
  
  timestep = timestep
  n_features = 39
  n_obs = timestep * n_features
  reframed = series_to_supervised(values, timestep, 1)
  reframed = reframed.iloc[: , :-38]

  values = reframed.values
  indice1 = train_h.shape[0]
  indice2 = valid_h.shape[0]

  train = values[:indice1, :]
  valid = values[indice1:indice1+indice2, :]
  test = values[indice1+indice2:, :]

  train_X, train_y = train[:, :-1], train[:, -1]
  valid_X, valid_y = valid[:, :-1], valid[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]

  if (resid_check==True):
    train_y = resid_train.values[:2184]
    valid_y = resid_train.values[2184:]

  scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_X)
  train_X = scaler.transform(train_X)
  valid_X = scaler.transform(valid_X)
  test_X = scaler.transform(test_X)

  scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(train_y.reshape(-1,1))
  train_y = scaler_y.transform(train_y.reshape(-1,1))
  valid_y = scaler_y.transform(valid_y.reshape(-1,1))
  test_y = scaler_y.transform(test_y.reshape(-1,1))

  train_X = train_X.reshape((train_X.shape[0], timestep, n_features))
  valid_X = valid_X.reshape((valid_X.shape[0], timestep, n_features))
  test_X = test_X.reshape((test_X.shape[0], timestep, n_features))
  return train_X, train_y, valid_X, valid_y, test_X, test_y, scaler, scaler_y

# WanDB

In [ ]:
!pip install wandb -qq
import wandb
from wandb.keras import WandbCallback
!wandb login

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 62.0 MB/s 
     |████████████████████████████████| 144 kB 70.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Train

# RNN

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'RNN',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'values': [3, 6, 12, 24]
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="DF1_SIMPLE_RNN")

Create sweep with ID: emf7xlnp
Sweep URL: https://wandb.ai/ancy/DF1_SIMPLE_RNN/sweeps/emf7xlnp


In [ ]:
def create_rnn():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y = make_data(hourly, timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(SimpleRNN(units = num_units, dropout=dropout, return_sequences=True))

      model.add(SimpleRNN(units = num_units, dropout=dropout))

    else:
      model.add(SimpleRNN(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])

    return model

In [ ]:
wandb.agent(sweep_id, create_rnn, count=50)

wandb: Agent Starting Run: etd1mkne with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 6
wandb: 	units: 128
wandb: Currently logged in as: ancy (use `wandb login --relogin` to force relogin)


epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃▁▂▄▆█▇▅▅▄▃▂
best_epoch,1
best_val_loss,0.08609
epoch,11
loss,0.09284
val_loss,0.09501


wandb: Agent Starting Run: mqe3su9k with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	layers: 2
wandb: 	lr: 0.1
wandb: 	timestep: 24
wandb: 	units: 256


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▄▃▃
best_epoch,20
best_val_loss,0.05698
epoch,30
loss,0.67416
val_loss,1.17481


wandb: Agent Starting Run: ojof7vel with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0
wandb: 	layers: 8
wandb: 	lr: 0.1
wandb: 	timestep: 24
wandb: 	units: 64


...

epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇▁▁▁█▆▃▅▅▆▅▅▁
best_epoch,2
best_val_loss,0.02817
epoch,12
loss,0.02236
val_loss,0.03149


wandb: Agent Starting Run: jsoqjly2 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 6
wandb: 	units: 128


epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▄▂▂▃▂▂▁▂▂▁▁▂▂▁▁▂▂
best_epoch,8
best_val_loss,0.02597
epoch,18
loss,0.02839
val_loss,0.03071


wandb: Agent Starting Run: kurzawki with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 6
wandb: 	units: 256


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▃▅██▅▂▂▂▂
best_epoch,4
best_val_loss,0.08556
epoch,14
loss,0.09604
val_loss,0.08876


# LSTM

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'LSTM',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'values': [3, 6, 12, 24]
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="DF1_SIMPLE_LSTM")

Create sweep with ID: nek8fpp9
Sweep URL: https://wandb.ai/ancy/DF1_SIMPLE_LSTM/sweeps/nek8fpp9


In [ ]:
def create_lstm():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y = make_data(hourly, timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(LSTM(units = num_units, dropout=dropout, return_sequences=True))

      model.add(LSTM(units = num_units, dropout=dropout))

    else:
      model.add(LSTM(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )

    model.fit(train_X, train_y, batch_size=batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)])

    return model

In [ ]:
wandb.agent(sweep_id, create_lstm, count=50)

wandb: Agent Starting Run: ejvyx0yw with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.1
wandb: 	layers: 2
wandb: 	lr: 0.1
wandb: 	timestep: 24
wandb: 	units: 64


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▅▄▁▄▆▇▄▂▂▄▂▃▂
best_epoch,4
best_val_loss,0.02384
epoch,14
loss,0.0176
val_loss,0.0246


wandb: Agent Starting Run: dkew4hwg with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	layers: 1
wandb: 	lr: 0.1
wandb: 	timestep: 12
wandb: 	units: 64


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁
best_epoch,34
best_val_loss,0.01907
epoch,44
loss,0.01635
val_loss,0.02288


wandb: Agent Starting Run: dnmky7nl with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 3
wandb: 	units: 64


...

epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
loss,█▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂█▁▂▂▂▃▂▂▂▁▃▁
best_epoch,2
best_val_loss,0.0268
epoch,12
loss,0.01684
val_loss,0.02716


wandb: Agent Starting Run: ahut6ueg with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 3
wandb: 	units: 256


epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▂▁▁▂▁▁▁▁▁▁▂▂▁▁▁▁
best_epoch,7
best_val_loss,0.01992
epoch,17
loss,0.01768
val_loss,0.02197


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nzsvshgg with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	layers: 8
wandb: 	lr: 0.01
wandb: 	timestep: 24
wandb: 	units: 128


epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃▃▁▁▁▁▁▁▁▁▂▄▇█▇▅▃▃
best_epoch,7
best_val_loss,0.08537
epoch,17
loss,0.08204
val_loss,0.08633


# GRU

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'name': 'GRU',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [256, 128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'units': {
            'values': [64, 128, 256]
        },
        'timestep': {
            'values': [3, 6, 12, 24]
        },
        'lr': {
            'values': [1e-2, 1e-1]
        },
        'layers': {
            'values': [1, 2, 4, 8]
        },
        
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="DF1_SIMPLE_GRU")

Create sweep with ID: 24hx7eek
Sweep URL: https://wandb.ai/ancy/DF1_SIMPLE_GRU/sweeps/24hx7eek


In [ ]:
def create_gru():
    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'units': 64,
    'timestep': 3,
    'dropout': 0.3,
    'layers': 1,
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    batch_size = config.batch_size
    lr         = config.lr
    num_units  = config.units
    timestep   = config.timestep
    dropout    = config.dropout
    layers     = config.layers

    train_X, train_y, valid_X, valid_y, test_X, test_y = make_data(hourly, timestep, resid_check=False)
    model = Sequential()

    if layers > 1:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout, return_sequences=True))

      for i in range(layers-2):
        model.add(GRU(units = num_units, dropout=dropout, return_sequences=True))

      model.add(GRU(units = num_units, dropout=dropout))

    else:
      model.add(GRU(units = num_units, input_shape=(train_X.shape[1], train_X.shape[2]), dropout=dropout))

    model.add(Dense(units = 1))
    lr = config.lr
    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
    )
    
    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='val_loss',patience=5, min_lr=1e-6)])

    return model

In [ ]:
wandb.agent(sweep_id, create_gru, count=50)

wandb: Agent Starting Run: dusagay7 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.3
wandb: 	layers: 4
wandb: 	lr: 0.1
wandb: 	timestep: 12
wandb: 	units: 128


epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,6
best_val_loss,0.08558
epoch,16
loss,0.08125
val_loss,0.08563


wandb: Agent Starting Run: 2hp3f030 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	layers: 2
wandb: 	lr: 0.01
wandb: 	timestep: 3
wandb: 	units: 64


epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▄▂▅█▁▁▂▄▂▂▄▅▅▅▅▅
best_epoch,5
best_val_loss,0.02655
epoch,15
loss,0.0133
val_loss,0.05512


wandb: Agent Starting Run: vfpkne9o with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.1
wandb: 	layers: 8
wandb: 	lr: 0.1
wandb: 	timestep: 24
wandb: 	units: 128


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,92
best_val_loss,0.01522
epoch,102
loss,0.00802
val_loss,0.01523


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: u338ulgw with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.1
wandb: 	layers: 4
wandb: 	lr: 0.01
wandb: 	timestep: 12
wandb: 	units: 64
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃█▄▄▃▃▄▃▃▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂
best_epoch,28
best_val_loss,0.01541
epoch,38
loss,0.00812
val_loss,0.0171


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: hx1jw1ox with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.1
wandb: 	layers: 8
wandb: 	lr: 0.01
wandb: 	timestep: 12
wandb: 	units: 64


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▃▃▃▄▃▂▃▂▂▂▂▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂
best_epoch,45
best_val_loss,0.01448
epoch,55
loss,0.00673
val_loss,0.01728


# Transformer

In [ ]:
sweep_config = {
    'name': 'New_Transformer',
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'best_val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'batch_size': {
            'values': [128, 64, 32]
        },
        'dropout': {
            'values': [0, 0.1, 0.3]
        },
        'timestep': {
            'values': [3, 6, 12, 24]
        },
        'head_size': {
            'values': [256, 128, 512]
        },
        'n_heads': {
            'values': [2, 4, 8]
        },
        'ff_dim': {
            'values': [128,256]
        },
        'n_blocks': {
            'values': [2, 4, 8]
        },
        'embed_dim': {
            'values': [32, 64, 128]
        },
        'time_2_vec': {
            'values': [2,3,4]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="ancy", project="DF1_SIMPLE_TRANS")

Create sweep with ID: lhljcyws
Sweep URL: https://wandb.ai/ancy/DF1_SIMPLE_TRANS/sweeps/lhljcyws


In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.ffn = keras.Sequential( [layers.Dense(ff_dim, activation = "gelu"), layers.Dense(feat_dim),] )
        self.layernorm1 = layers.BatchNormalization()
        self.layernorm2 = layers.BatchNormalization()
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.embed_dim = embed_dim
        self.feat_dim = feat_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):

        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'feat_dim': self.feat_dim,
            'num_heads': self.num_heads,
            'ff_dim': self.ff_dim,
            'rate': self.rate,
        })
        return config

In [ ]:
class Time2Vec(keras.layers.Layer):
    def __init__(self, kernel_size = 1):
        super(Time2Vec, self).__init__(trainable = True, name = 'Time2VecLayer')
        self.k = kernel_size

    def build(self, input_shape):
        # trend
        self.wb = self.add_weight(name = 'wb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        self.bb = self.add_weight(name = 'bb', shape = (input_shape[1],), initializer = 'uniform', trainable = True)
        # periodic
        self.wa = self.add_weight(name = 'wa', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        self.ba = self.add_weight(name = 'ba', shape = (1, input_shape[1], self.k), initializer = 'uniform', trainable = True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs, **kwargs):
        bias = self.wb * inputs + self.bb
        dp = K.dot(inputs, self.wa) + self.ba
        wgts = K.sin(dp) # or K.cos(.)
        ret = K.concatenate([K.expand_dims(bias, -1), wgts], -1)
        ret = K.reshape(ret, (-1, inputs.shape[1] * (self.k + 1)))
        return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * (self.k + 1))

    def get_config(self):

        config = super().get_config()
        config.update({
            'kernel_size': self.k,
        })
        return config

In [ ]:
EPOCHS = 50
N_HEADS = 8
N_FOLDS = 10
FF_DIM = 256
N_BLOCKS = 6
EMBED_DIM = 64
BATCH_SIZE = 16
WINDOW_SIZE = 65
DROPUT_RATE = 0.0
TIME_2_VEC_DIM = 3
TRAIN_MODEL = True
SKIP_CONNECTION_STRENGTH = 0.9

In [ ]:
def get_model(input_shape, time2vec_dim = 3):
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "selu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)
    return model

In [ ]:
def get_model2():

    config_defaults = {
    'batch_size': 128,
    'lr': 1e-3,
    'timestep': 3,
    'n_heads': 8,
    'ff_dim':256,
    'n_blocks':6,
    'embed_dim':64,
    'dropout':0,
    'time_2_vec':3,
    }

    wandb.init(config=config_defaults)
    config = wandb.config
    train_X, train_y, valid_X, valid_y, test_X, test_y = make_data(hourly, config.timestep, resid_check=False)


    batch_size=config.batch_size
    lr=config.lr
    N_HEADS = config.n_heads
    FF_DIM = config.ff_dim
    N_BLOCKS = config.n_blocks
    EMBED_DIM = config.embed_dim
    DROPUT_RATE = config.dropout
    time2vec_dim = config.time_2_vec

    input_shape = train_X.shape[1:]
    inp = Input(input_shape)
    x = inp

    time_embedding = keras.layers.TimeDistributed(Time2Vec(time2vec_dim - 1))(x)
    x = Concatenate(axis = -1)([x, time_embedding])
    x = layers.LayerNormalization(epsilon = 1e-6)(x)

    for k in range(N_BLOCKS):
        x_old = x
        transformer_block = TransformerBlock(EMBED_DIM, input_shape[-1] + ( input_shape[-1] * time2vec_dim), N_HEADS, FF_DIM, DROPUT_RATE)
        x = transformer_block(x)
        x = ((1.0 - SKIP_CONNECTION_STRENGTH) * x) + (SKIP_CONNECTION_STRENGTH * x_old)

    x = layers.Flatten()(x)

    x = layers.Dense(128, activation = "relu")(x)
    x = layers.Dropout(DROPUT_RATE)(x)
    x = Dense(1, activation = 'linear')(x)

    out = x
    model = Model(inp, out)

    model.compile(
        loss="mse",
        optimizer=Adam(learning_rate=lr)
                  )

    model.fit(train_X, train_y, batch_size=config.batch_size,
              epochs=200, verbose=0, shuffle=False,
              validation_data=(valid_X, valid_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=10, restore_best_weights=True)]
              )
    
    return model

In [ ]:
wandb.agent(sweep_id, get_model2, count=50)

wandb: Agent Starting Run: 39tdpjkz with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0
wandb: 	embed_dim: 64
wandb: 	ff_dim: 256
wandb: 	head_size: 512
wandb: 	n_blocks: 8
wandb: 	n_heads: 8
wandb: 	time_2_vec: 2
wandb: 	timestep: 6
wandb: Currently logged in as: ancy (use `wandb login --relogin` to force relogin)


epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
loss,█▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▂▅▄▅▅▃▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,12
best_val_loss,0.03308
epoch,22
loss,0.0059
val_loss,0.03467


wandb: Agent Starting Run: q2ic93i6 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	embed_dim: 32
wandb: 	ff_dim: 128
wandb: 	head_size: 256
wandb: 	n_blocks: 4
wandb: 	n_heads: 8
wandb: 	time_2_vec: 2
wandb: 	timestep: 6


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,19
best_val_loss,0.04606
epoch,29
loss,0.01022
val_loss,0.06033


wandb: Agent Starting Run: q2c9fso4 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	embed_dim: 32
wandb: 	ff_dim: 128
wandb: 	head_size: 256
wandb: 	n_blocks: 2
wandb: 	n_heads: 8
wandb: 	time_2_vec: 4
wandb: 	timestep: 12


...

epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▂▁▁▁▁▂▂▂▁▂▃▃▂
best_epoch,22
best_val_loss,0.01539
epoch,32
loss,0.0384
val_loss,0.01956


wandb: Agent Starting Run: 8c54v8b1 with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	embed_dim: 128
wandb: 	ff_dim: 256
wandb: 	head_size: 128
wandb: 	n_blocks: 2
wandb: 	n_heads: 2
wandb: 	time_2_vec: 2
wandb: 	timestep: 24


epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅█▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,21
best_val_loss,0.01825
epoch,31
loss,0.04173
val_loss,0.02246


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bry6lgvn with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	embed_dim: 32
wandb: 	ff_dim: 256
wandb: 	head_size: 512
wandb: 	n_blocks: 2
wandb: 	n_heads: 2
wandb: 	time_2_vec: 2
wandb: 	timestep: 24


epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▇▆▅█▇▅▃▅▂▂▂▁▂▄▁▁▁▃▃▂▁▂▂▁▁▃▃▂▁▅▄▂▃▃
best_epoch,25
best_val_loss,0.01857
epoch,35
loss,0.02048
val_loss,0.02605
